## Error Analysis

We use the finegrained task  here



In [1]:
%load_ext autoreload
%autoreload 2

from hatedetection import load_datasets
import json

train_dataset, dev_dataset, test_dataset = load_datasets(add_body=True)

INFO: Pandarallel will run on 24 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


Obs

`text` hace referencia al contexto del tweet del diario
`title` al título de la noticia (extraído del artículo)

In [2]:


evaluations = {
}
for model_name, path in [
    ('none-ft', "../evaluations/betonews_fine_none.json"),   
    ('text-ft', "../evaluations/betonews_fine_text.json"),   
    ('body-ft', "../evaluations/betonews_fine_body.json"),   
    ]:

    with open(path) as f:
        evaluations[model_name] = json.load(f)

for key, evals in evaluations.items():
    print(f"We have {len(evals['metrics'])} {key} evaluations")


We have 10 none-ft evaluations
We have 10 text-ft evaluations
We have 15 body-ft evaluations


In [12]:
import numpy as np
from hatedetection import extended_hate_categories

df = test_dataset.to_pandas().set_index("id")
df["HATEFUL"] = df[extended_hate_categories].any(1)

    



context_column_names = [x+"_context" for x in extended_hate_categories]
none_column_names = [x+"_none" for x in extended_hate_categories]

df[context_column_names] = (np.array(evaluations["text-ft"]["predictions"]) > 0).mean(0)
df["HATEFUL_context"] = (df[context_column_names] > 0.5).any(1)

df[none_column_names] = (np.array(evaluations["none-ft"]["predictions"]) > 0).mean(0)
df["HATEFUL_none"] = (df[none_column_names] > 0.5).any(1)

"""
Obtain offended chars for each row
"""
def get_chars(row):    
    return [k.split("_")[0] for k, v in row.items() if v > 0.5]

df["CHARS_none"] = df[none_column_names].apply(get_chars, axis=1)
df["CHARS_context"] = df[context_column_names].apply(get_chars, axis=1)

df.describe()

,CALLS,WOMEN,LGBTI,RACISM,CLASS,POLITICS,DISABLED,APPEARANCE,CRIMINAL,CALLS_context,...,CRIMINAL_context,CALLS_none,WOMEN_none,LGBTI_none,RACISM_none,CLASS_none,POLITICS_none,DISABLED_none,APPEARANCE_none,CRIMINAL_none
count,11343.000000,11343.000000,11343.000000,11343.000000,11343.000000,11343.000000,11343.000000,11343.000000,11343.000000,11343.000000,...,11343.000000,11343.000000,11343.000000,11343.000000,11343.000000,11343.000000,11343.000000,11343.000000,11343.000000,11343.000000
mean,0.024861,0.023980,0.017191,0.042758,0.013312,0.025655,0.011373,0.033413,0.031914,0.020753,...,0.026148,0.020709,0.013841,0.008437,0.039954,0.010853,0.019527,0.010756,0.032099,0.023239
std,0.155709,0.152992,0.129989,0.202319,0.114613,0.158110,0.106039,0.179720,0.175779,0.133412,...,0.150197,0.132521,0.100109,0.079997,0.181947,0.090870,0.127778,0.095530,0.167408,0.136992
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## CRIMINAL

In [16]:
import pandas as pd

cat = "CRIMINAL"

pd.options.display.max_rows = None
pd.options.display.max_colwidth = 240
df[df[cat] & df["HATEFUL_context"] & ~df["HATEFUL_none"] ].loc[:, ["article_text", "text", cat+"_context", cat+"_none", "CHARS_context"]]

,article_text,text,CRIMINAL_context,CRIMINAL_none,CHARS_context
id,,,,,
358021,El polémico cortejo del ladrón asesinado por el jubilado en Quilmes url url,@usuario Le hubieran puesto una bomba al cortejo,1.0,0.0,"[CALLS, CRIMINAL]"
358068,El polémico cortejo del ladrón asesinado por el jubilado en Quilmes url url,@usuario A nadie se le ocurrió poner una bomba ahí?,1.0,0.0,"[CALLS, CRIMINAL]"
404726,"Usurpación en Cariló. Cumbia y champagne, así festejaban en la casa tomada url url",@usuario UNO NO SABE SI REIR O LLORAR... QUE GRASA MAS GRASA... A LOS TIROS HAY QUE SACARLOS,0.7,0.2,[CRIMINAL]
405653,Así quedó la iglesia de El Bolsón vandalizada por mapuches url,@usuario BASTA DE ESTOS SINVERGÜENZAS...ESPERO QUE LOS METAN TODOS PRESOS Y LE SAQUEN LOS PLANES SOCIALES!!!,0.0,0.1,"[CALLS, RACISM, CLASS]"
407351,El asesinato del ciudadano armenio reavivó la discusión sobre la imputabilidad de menores url,@usuario Que discusión? Bala,1.0,0.3,"[CALLS, CRIMINAL]"
364818,Una madre tuvo que esconder a su hija porque su vecino violador con prisión domiciliaria la acosa constantemente | Por Miguel Prieto Toledo url,@usuario Pegale un tiro a ese emoji anciano tono de piel oscuro medio emoji hdrp emoji cara con símbolos en la boca emoji emoji cara con símbolos en la boca emoji emoji cara con símbolos en la boca emoji el padre de la nena un tio alg...,0.9,0.2,[CRIMINAL]
370382,Una mujer policía de la Ciudad mató a balazos a un delincuente que robaba ruedas de autos url,@usuario Denle un aplauso a esa buena mujer ! emoji manos aplaudiendo emoji emoji manos aplaudiendo emoji emoji manos aplaudiendo emoji,1.0,0.0,[CRIMINAL]
370383,Una mujer policía de la Ciudad mató a balazos a un delincuente que robaba ruedas de autos url,"@usuario La felicito ,es más le doy un ascenso ya",0.7,0.0,[CRIMINAL]
370391,Una mujer policía de la Ciudad mató a balazos a un delincuente que robaba ruedas de autos url,@usuario Genia! La mujeres q todos qremos.,0.7,0.0,[CRIMINAL]


| Clas. no contextual | Contexto          |           Texto                                                                                         |
|-----|-----------------------------|---------------------------------------------------------------------------------------------------------|
| FN    |Una policía baleó y mató a un joven de 17 años que la atacó con una tijera en Moreno url	| bellezaaa                                      |
| FN                                                                                          | 👏 excelente!                                  |
| FN |El polémico cortejo del ladrón asesinado por el jubilado en Quilmes | Le hubieran puesto una bomba al cortejo                              |
| FP | Ivana Nadal se cansó de la criticaran y sorprendió con su respuesta: "Gracias a Dios a fin de año me voy del país" | Esa si es una buena noticia.	|
| FP | Mora Godoy cierra su escuela de tango y remata su vestuario para "poder seguir adelante" | Adoro los finales felices |
| FP |¿Se va del país? Juana Viale estaría tramitando la ciudadanía uruguaya url url	@usuario Q bueno ? Una mierda menos . Q se quede alla |


¿Y los falsos positivos por el no contextualizado que ahora desechamos?

In [26]:
cat = "CRIMINAL"

pd.options.display.max_rows = None
pd.options.display.max_colwidth = 240
df[~df["HATEFUL"] & (df[cat+"_none"] > 0.5) & ~df["HATEFUL_context"] ].loc[:, ["article_text", "text", cat+"_context", cat+"_none", "CHARS_none", "CHARS_context"]]

,article_text,text,CRIMINAL_context,CRIMINAL_none,CHARS_none,CHARS_context
id,,,,,,
397715,"Mora Godoy cierra su escuela de tango y remata su vestuario para ""poder seguir adelante"" url",@usuario Lo siento oero me alegro eso se llama Karma,0.0,1.0,[CRIMINAL],[]
397738,"Mora Godoy cierra su escuela de tango y remata su vestuario para ""poder seguir adelante"" url",@usuario Adoro los finales felices.,0.0,1.0,[CRIMINAL],[]
358036,El polémico cortejo del ladrón asesinado por el jubilado en Quilmes url url,"@usuario Ya está muerto, su familia que tantooo lo va a extrañar que le enseñó? Que educación le dió? \nLa viudita le dijo alguna vez no salgas a robar, tenés un hijo, buscá laburo? \nNOOO, por lo tanto son todos responsables de está mu...",0.0,0.6,[CRIMINAL],[]
387335,"Ivana Nadal se cansó de la criticaran y sorprendió con su respuesta: ""Gracias a Dios a fin de año me voy del país"" url",@usuario Esa si es una buena noticia.,0.0,1.0,[CRIMINAL],[]
355773,¿Se va del país? Juana Viale estaría tramitando la ciudadanía uruguaya url url,@usuario Q bueno ? Una mierda menos . Q se quede allá,0.0,0.9,[CRIMINAL],[]
404702,"Usurpación en Cariló. Cumbia y champagne, así festejaban en la casa tomada url url",@usuario Libre portación de armas...,0.3,1.0,"[CALLS, CRIMINAL]",[]
330834,"coronavirus - Mauricio Macri llamó ayer por teléfono a Alberto Fernández\n\nSegún trascendidos, el ex presidente no está de acuerdo con la paralización total del país. \n\nMás información emoji flecha hacia abajo emoji \n\nurl","@usuario @usuario El expresidente mafioso y ladròn se merece en realidad la horca,que cuelgue de su cuello hasta que muera,,o algo asì,,serìa justicia,para un ladròn y asesino de pibes y viejos",0.0,0.9,"[CALLS, CRIMINAL]",[]
373146,El Presidente encabezó una videoconferencia en donde anticipó que la semana que viene anunciará medidas para combatir el delito en la provincia de Buenos Aires url url,@usuario Matate,0.0,0.8,[CRIMINAL],[]
336621,"Una mujer que violó la cuarentena mordió y les arrojó gas pimienta a tres policías: ""No es estado de sitio, no me podés tocar, un decreto no está por encima de la Constitución"" url",@usuario No entiendo es una mujer o es un animal rabioso que muerde. Que la pongan entre rejas así no sigue mordiendo y FUERTE MULTA que podría servir para ayudar a los necesitados de alimentos @usuario @usuario,0.1,0.7,[CRIMINAL],[]


## LGBTI


In [17]:
import pandas as pd

cat = "LGBTI"

pd.options.display.max_rows = None
pd.options.display.max_colwidth = 240

df[df[cat] & df["HATEFUL_context"] & ~df["HATEFUL_none"] ].loc[:, ["article_text", "text", cat+"_context", cat+"_none", "CHARS_context"]]

,article_text,text,LGBTI_context,LGBTI_none,CHARS_context
id,,,,,
375847,"Contó que era lesbiana, su papá le confesó que era gay y ahora su madre se enamoró de una mujer, lo que inspiró su segundo film | Por Hugo Martin\nurl","@usuario Revisen esa casa, los están envenenando.",0.6,0.0,[LGBTI]
375859,"Contó que era lesbiana, su papá le confesó que era gay y ahora su madre se enamoró de una mujer, lo que inspiró su segundo film | Por Hugo Martin\nurl","@usuario El hermano se coge al perro, no?",0.6,0.0,[LGBTI]
375872,"Contó que era lesbiana, su papá le confesó que era gay y ahora su madre se enamoró de una mujer, lo que inspiró su segundo film | Por Hugo Martin\nurl",@usuario emoji cara vomitando emoji emoji cara vomitando emoji emoji cara vomitando emoji,0.9,0.0,[LGBTI]
398893,Mara Gómez: la historia de la primera futbolista trans en el torneo argentino url url,@usuario Igual deberá visitar al médico para revisión de próstata emoji cara pensativa emoji,0.6,0.0,[LGBTI]
398905,Mara Gómez: la historia de la primera futbolista trans en el torneo argentino url url,"@usuario Bien Roberto, lo lograste: en un equipo de tipos no tenias chancho, pero le buscaste la vuelta y llegaste... Pais de emprendedores! emoji cara sonriendo con los ojos cerrados emoji",0.6,0.0,[LGBTI]
398912,Mara Gómez: la historia de la primera futbolista trans en el torneo argentino url url,@usuario Tiene más frente de la Casa Blanca \nEsta quedando pelado el guacho,0.6,0.0,[LGBTI]
398917,Mara Gómez: la historia de la primera futbolista trans en el torneo argentino url url,@usuario A romper piernas muchacho,1.0,0.0,[LGBTI]
398920,Mara Gómez: la historia de la primera futbolista trans en el torneo argentino url url,"@usuario Mara ""garrote"" Gómez",0.8,0.0,[LGBTI]
398925,Mara Gómez: la historia de la primera futbolista trans en el torneo argentino url url,"@usuario Sigue siendo HOMBRE, que por GENÉTICA, no por una ideología u orientacion sexual, GE-NÉ-TI-CA, es más fuerte que la mujer.",1.0,0.3,[LGBTI]


In [27]:
cat = "LGBTI"

pd.options.display.max_rows = None
pd.options.display.max_colwidth = 240
df[~df["HATEFUL"] & (df[cat+"_none"] > 0.5) & ~df["HATEFUL_context"] ].loc[:, ["article_text", "text", cat+"_context", cat+"_none", "CHARS_none", "CHARS_context"]]

,article_text,text,LGBTI_context,LGBTI_none,CHARS_none,CHARS_context
id,,,,,,
358040,El polémico cortejo del ladrón asesinado por el jubilado en Quilmes url url,@usuario Ahora sí no te vas a hacer más el macho pegándole a los viejos. ¡Hijo de mil puta!,0.0,0.6,[LGBTI],[]
353807,URGENTE: Un hombre se incrustó con su auto en la puerta de la Embajada de China y aseguró que tenía explosivos url,@usuario No es hombre . Es un boludo,0.0,1.0,[LGBTI],[]
395329,"El exabrupto de Jair Bolsonaro sobre Brasil y el Covid-19: ""Tienen que dejar de ser un país de maricas"" url",@usuario Que capo!,0.0,0.6,[LGBTI],[]
375840,"Contó que era lesbiana, su papá le confesó que era gay y ahora su madre se enamoró de una mujer, lo que inspiró su segundo film | Por Hugo Martin\nurl",@usuario Una mujer descendiente de una vida de mentiras y apariencias ...una mujer que su ascendencia usó para justificar y ocultar su mundo ...otra mujer simplemente usada ...aún si me dicen que sus padres son ambisexuales y no lesbian...,0.1,1.0,[LGBTI],[]
384377,"China: identificaron otro virus ""con potencial para convertirse en pandemia"" nuevo virus china transportado por cerdos podría infectar ahumanos las actuales vacunas podrían adaptarse más en la nota url",@usuario Los TROLL M preocupadop,0.3,0.9,[LGBTI],[]
406503,La aberrante arenga machista que dio el preparador físico de Güemes de Santiago del Estero url,@usuario dejen de mariconear \npelotudos inviables,0.0,0.7,[LGBTI],[]
362970,"Denuncian que Ronaldinho pasa sus noches de fiesta con modelos durante su prisión domiciliaria: ""Hacen bailes hasta horas de la madrugada"" url",@usuario El Hombre,0.0,1.0,[LGBTI],[]
392010,Coronavirus: Ginés reveló cuál fue la palabra que detonó el acuerdo con Pfizer url url,@usuario Sale un pete?,0.0,0.6,[LGBTI],[]
344821,Coronavirus en Argentina: el 70% de los casos son en hombres url,@usuario Es femenino el Corona...,0.2,0.9,[LGBTI],[]


| Error          |Contexto                         |           Texto                                                                                         |
|----------|------------------------|---------------------------------------------------------------------------------------------------------|
| FN             |"Te amo ". La emotiva dedicatoria de Luis Novaresio a su pareja en su cumpleaños | Definitivamente no acepto esta degeneración repugnante de la humanidad. |
| FN             | Mara Gómez: la historia de la primera futbolista trans en el torneo argentino | Sigue siendo HOMBRE, que por GENÉTICA, no por una ideología u orientacion sexual, GE-NÉ-TI-CA, es más fuerte que la mujer. |
| FN             | Contó que era lesbiana, su papá le confesó que era gay y ahora su madre se enamoró de una mujer, lo que inspiró su segundo film| Revisen esa casa, los están envenenando.|
| FP      | URGENTE: Un hombre se incrustó con su auto en la puerta de la Embajada de China y aseguró que tenía explosivos | No es hombre . Es un boludo	|
| FP    | Detuvieron al hombre que admitió violar a su hija en audios de WhatsApp y fue tendencia en redes: un tío lo entregó | Degenerado MP emoji cara vomitando emoji |

## CLASS

In [18]:
import pandas as pd

cat = "CLASS"

pd.options.display.max_rows = None
pd.options.display.max_colwidth = 240

df[df[cat] & df["HATEFUL_context"] & ~df["HATEFUL_none"] ].loc[:, ["article_text", "text", cat+"_context", cat+"_none", "CHARS_context"]]

,article_text,text,CLASS_context,CLASS_none,CHARS_context
id,,,,,
398859,"Empujones, corridas y botellazos en la despedida a Diego Maradona url",@usuario soltás a los negritos 2 minutos y se matan.\no bien festejan de la única forma que saben\ndespués el criminal es el que pide abrir las escuelas.,0.0,0.1,[RACISM]
343209,"En medio de la cuarentena por el coronavirus, el Gobierno avanza con un impuesto a los abonos de Netflix url",@usuario ESTE TÍTERE ES LO MÁS HDRMP...TODAVÍA QUE MANTENEMOS A SUS PARÁSITOS K Y PLANEROS QUIERE SACARNOS MÁS PLATA...\nTITERE CON YEGUA LINDA JUNTA!!!,0.4,0.1,[POLITICS]
334366,"Juan Grabois: ""Hay que aplicar la autoridad sobre las personas que se manifestaron"" url",@usuario Y bala para los que toman tierras ajenas y para los que lo promueven !! Cuidado grabois ahí va una !!!,0.9,0.5,"[CALLS, CLASS, POLITICS]"
405653,Así quedó la iglesia de El Bolsón vandalizada por mapuches url,@usuario BASTA DE ESTOS SINVERGÜENZAS...ESPERO QUE LOS METAN TODOS PRESOS Y LE SAQUEN LOS PLANES SOCIALES!!!,0.8,0.5,"[CALLS, RACISM, CLASS]"
405013,Otro día de marchas y cortes en el centro porteño: 3 caravanas se movilizan en apoyo al Gobierno url,"@usuario Vagos, lacras!!!",1.0,0.1,[CLASS]
357690,"El mensaje de El Dipy que lo convirtió en tendencia en las redes: ""Piensan que porque salí de un barrio humilde, soy kirchnerista o soy peronista"" url","@usuario Jjaja, no entras en ninguna categoría papá!!! Ahhh y hacete bien esos dientes dracu dracu que tenes! Otario",0.0,0.0,[APPEARANCE]
392268,"Coronavirus: Alberto Fernández advirtió que, 'si el relajamiento sigue, las fuerzas de seguridad saldrán a la calle para disipar a la gente' url",@usuario emoji cara revolviéndose de la risa emoji emoji cara revolviéndose de la risa emoji emoji cara revolviéndose de la risa emoji siii hacerme reor payaso esto l9 hiciste vos d3l8ncuente con un velorio y piquete de los parasitos...,0.9,0.5,"[CLASS, POLITICS]"
401976,La temporada de verano en la Costa Atlántica empezó con un corte total en la Ruta 2: organizaciones sociales piden canastas navideñas url,@usuario @usuario Jjaja emoji cara con mano sobre la boca emoji emoji cara revolviéndose de la risa emoji emoji cara revolviéndose de la risa emoji emoji cara revolviéndose de la risa emoji siempre los vividores,0.8,0.2,[CLASS]
401977,La temporada de verano en la Costa Atlántica empezó con un corte total en la Ruta 2: organizaciones sociales piden canastas navideñas url,@usuario Pais bananero LABUREN lpm,0.9,0.3,[CLASS]


In [28]:
cat = "CLASS"

pd.options.display.max_rows = None
pd.options.display.max_colwidth = 240
df[~df["HATEFUL"] & (df[cat+"_none"] > 0.5) & ~df["HATEFUL_context"] ].loc[:, ["article_text", "text", cat+"_context", cat+"_none", "CHARS_none", "CHARS_context"]]

,article_text,text,CLASS_context,CLASS_none,CHARS_none,CHARS_context
id,,,,,,
389699,Pablo Moyano amenazó con un paro en la Ciudad de Buenos Aires si hay recortes en la recolección de basura url,@usuario Anda a laburar!!!,0.0,1.0,[CLASS],[]
384114,Durísimo descargo de Migue Granados luego de que le pidieran 9 mil dólares por mes para alquilar una casa durante el verano url,"@usuario x lo visto usas un usd de mas de 150 rey coff coff, veraneas en argentina, q mas querés... \nayudas a llenarle la heladera a otro. ademas seguro te dá factura. con esos impuestos se paga el IFE, los terrenos de los okupas, el p...",0.0,0.8,[CLASS],[]
352199,El Gobierno destinará 75 mil millones de pesos para urbanizar 300 villas en todo el país | Por Andrés Klipphan\nurl,@usuario De darles trabajo ni hablemos no?,0.0,1.0,[CLASS],[]
352220,El Gobierno destinará 75 mil millones de pesos para urbanizar 300 villas en todo el país | Por Andrés Klipphan\nurl,"@usuario Me parece la mejor forma de terminar con los barrios de emergencia, mal llamados villas: urbanizándolos.",0.4,0.7,[CLASS],[]
407573,"Alberto Fernández: ""Es imperdonable que haya indocumentados en Argentina "" url url","@usuario De paso, el el gobernador que vos admirás, tiene una frontera que es un colador. No te contó que entran a cobrar AUH y planes? Eso es más piorrr. No viven en Argentina. Te preocupan los indocumentados y no cómo manejás el país....",0.0,1.0,"[RACISM, CLASS]",[]
336841,"Coronavirus en la Argentina: ""Es un error abrir más. No lo puedo permitir"", advirtió Axel Kicillof url url","@usuario Si este infeliz es el gobernador, porque no visita las tierras tomadas donde viven en total mugre e indigencia sin agua y baños de allí salen los machos virus..",0.0,0.6,[CLASS],[]
335586,Coronavirus en la Argentina: El Presidente recibió a las organizaciones sociales y prometió fondos para la economía popular url url,"@usuario Y de laburar, ni hablar!!",0.2,1.0,[CLASS],[]
388160,[OPINIÓN] En 2018 se registró la tasa más baja de natalidad de las últimas décadas y el tema no parece importarles a nuestros dirigentes | Por Claudia Peiró url,@usuario ¿Qué no era que tienen hijos para cobrar planes sociales? No entiendo...,0.0,0.7,[CLASS],[]
388169,[OPINIÓN] En 2018 se registró la tasa más baja de natalidad de las últimas décadas y el tema no parece importarles a nuestros dirigentes | Por Claudia Peiró url,@usuario Traer hijos a este País no sirve,0.0,0.6,[CLASS],[]


|   | Contexto                         |           Texto                                                                                         |
|---|----------------------------------|---------------------------------------------------------------------------------------------------------|
|FN | Las organizaciones sociales salieron al cruce de la acusación de Sergio Berni por la toma de tierras: "La falta de vivienda no se resuelve con balas" | Si, los podemos cagar matando a todos y listo
|FN |                                       | @usuario Dale alas y planes a los cuervos y veras cómo te sacan los ojos |
|FN | La temporada de verano en la Costa Atlántica empezó con un corte total en la Ruta 2: organizaciones sociales piden canastas navideñas | Hay que desparasitar urgente el país.	
|FP | Javier Milei confirmó que va "a militar en política" junto a José Luis Espert para que "en 35 años la Argentina sea primera potencia mundial" | Otro parásito	|
|FP | Leche de cucaracha, la nueva bebida nutritiva: ¿quién se anima a probarla? | Uds pueden producir litros, son todes cucarachas y ratas!!! |
|FP |Martha Rosenberg: "En situación de pandemia, legalizar el aborto es más urgente que nunca" | La verdad que sí...así se dejan de reproducir!!! |


## RACISM

In [22]:
import pandas as pd

cat = "RACISM"

pd.options.display.max_rows = None
pd.options.display.max_colwidth = 240

df[df[cat] & df["HATEFUL_context"] & ~df["HATEFUL_none"] ].loc[:, ["article_text", "text", cat+"_context", cat+"_none", "CHARS_context"]].sort_values("RACISM_context", ascending=False)

,article_text,text,RACISM_context,RACISM_none,CHARS_context
id,,,,,
393060,China lanzó una peligrosa venganza por los reportes sobre la eficacia de una de sus vacunas url,@usuario El pais mas mugriento y mentiroso que podemos esperar de esa gente que solo le importa prevalecer ante todo.,1.0,0.2,[RACISM]
405653,Así quedó la iglesia de El Bolsón vandalizada por mapuches url,@usuario BASTA DE ESTOS SINVERGÜENZAS...ESPERO QUE LOS METAN TODOS PRESOS Y LE SAQUEN LOS PLANES SOCIALES!!!,1.0,0.0,"[CALLS, RACISM, CLASS]"
393815,"Un refugio para animales en China habría vendido pieles de tigres en peligro de extinción y hecho un vino ""afrodisíaco"" con sus huesos url",@usuario COSTUMBRES Y CULTURAS CHINAS SON REPUDIABLES Y PARA NUNCA IMITAR.,1.0,0.5,[RACISM]
393808,"Un refugio para animales en China habría vendido pieles de tigres en peligro de extinción y hecho un vino ""afrodisíaco"" con sus huesos url",@usuario Cbinos de mierda! No hace 1 como gente!,1.0,0.2,[RACISM]
403672,Tensión en Villa Mascardi: mapuches resisten el desalojo de un predio usurpado url,@usuario Patada en el ogt a los ladrones !,1.0,0.0,"[CALLS, RACISM]"
403633,Tensión en Villa Mascardi: mapuches resisten el desalojo de un predio usurpado url,"@usuario Con el mismo dialogo podriamos decir que las malvinas realmente son inglesas porque un ancestro se los dijo, no? HAY QUE SACARLOS A TIROS A ESTOS HIJOS DE PUTA",1.0,0.5,"[CALLS, RACISM]"
354155,Alarma en China: más de 3000 personas se enfermaron tras la fuga de una bacteria url url,@usuario Suspendan china,1.0,0.3,"[CALLS, RACISM]"
354171,Alarma en China: más de 3000 personas se enfermaron tras la fuga de una bacteria url url,@usuario Brutos asesinos,1.0,0.0,[RACISM]
364665,Alertan por un nuevo virus en China con potencial de pandemia\nlo tienen los cerdos similar ala gripe porcina pero peor pará un poco2020 una pandemia es demasiado dos no se aguantan china lo hizo de nuevo yahora qué? estamos perdirijill...,@usuario @usuario Lo que sucede es que todos los CHINOS SON MUGRIENTOS.,1.0,0.4,[RACISM]


|     | Contexto                         |           Texto                                                                                         |
|-----|----------------------------------|---------------------------------------------------------------------------------------------------------|
|FN   | La pregunta billonaria: ¿China debería pagar el costo de la pandemia? | Si obviamente y desaparecer de la faz de la tierra. Mira el quilombo que armó. Se nos están muriendo todos...	|
|FN   | China: Vuelven a vender murciélagos y carne de animales domésticos a pesar del Coronavirus |  @usuario EEUU haceme limpieza ahí.	|
|FN   |Tensión en Villa Mascardi: mapuches resisten el desalojo de un predio usurpado | Era mas facil si usaban napalm |
|FP   | Confusión por la foto de Alberto Fernández con activistas veganos en medio de las negociaciones con China por el acuerdo porcino | Quilombo en puerta con China, son un desastre |
|FP   | Cristina Kirchner anunció que regresa con su hija Florencia a la Argentina | Es hora de actuar bichito chino !!... no nos podes fallar es un combo 2 x 1 |
|FP   | La aberrante arenga machista que dio el preparador físico de Güemes de Santiago del Estero | Un capo. |


In [31]:
cat = "RACISM"

pd.options.display.max_rows = None
pd.options.display.max_colwidth = 240
df[~df["HATEFUL"] & (df[cat+"_none"] > 0.5) & ~df["HATEFUL_context"] ].loc[:, ["article_text", "text", cat+"_context", cat+"_none", "CHARS_none", "CHARS_context"]].sort_values("RACISM_context", ascending=True)

,article_text,text,RACISM_context,RACISM_none,CHARS_none,CHARS_context
id,,,,,,
354905,"""Tucán redimido"": El duro descalificativo de Maximiliano Guerra hacia Florencia Kirchner url url",@usuario Geniooo,0.0,0.9,[RACISM],[]
354891,"""Tucán redimido"": El duro descalificativo de Maximiliano Guerra hacia Florencia Kirchner url url",@usuario @usuario Geniooo,0.0,0.8,[RACISM],[]
389984,La caravana en homenaje a Néstor Kirchner llegó a la Plaza de Mayo url,@usuario Esta plaga no la puedo ni ver!!!,0.0,0.6,[RACISM],[]
406500,La aberrante arenga machista que dio el preparador físico de Güemes de Santiago del Estero url,@usuario Un capo.,0.0,1.0,[RACISM],[]
403650,Tensión en Villa Mascardi: mapuches resisten el desalojo de un predio usurpado url,"@usuario Zona controlada x mapuches, usurpada x delincuentes emoji mano con señal de victoria emoji emoji mano con señal de victoria emoji ya están presos?",0.0,0.8,[RACISM],[]
403668,Tensión en Villa Mascardi: mapuches resisten el desalojo de un predio usurpado url,"@usuario Mi vieja: ayyy esos mapuches, que nisiquiera son mapuches! \nJAJAJA ESO ES VERDAD, la mayoria no lo son, son pocos los que son Mapuches MAPUCHES, los que quedan... emoji cara sonriendo con sudor frío emoji",0.0,0.8,[RACISM],[]
357707,"El mensaje de El Dipy que lo convirtió en tendencia en las redes: ""Piensan que porque salí de un barrio humilde, soy kirchnerista o soy peronista"" url",@usuario Geniooo,0.0,0.9,[RACISM],[]
391745,Preocupante hallazgo en China: detectaron coronavirus en helados url,@usuario jsjsj man o son re sucios o nos están boludeando,0.0,0.6,[RACISM],[]
392241,"Coronavirus: Alberto Fernández advirtió que, 'si el relajamiento sigue, las fuerzas de seguridad saldrán a la calle para disipar a la gente' url",@usuario Èsssta de acá !!!,0.0,0.7,[RACISM],[]
